In [1]:
import numpy as np
import pandas as pd
import warnings
import datetime as dt
import os
import matplotlib.pyplot as plt
from matplotlib.axes import Axes

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# common
color_style_dict = {
    '国网电动': '#0D8C8F',
    '南网电动': '#00367A',
    '特来电': '#00A6CA',
    '星星充电': '#F0830C',
    '阿维塔': '#000000',
}

def get_color_style(key):
    return color_style_dict.get(key, '#000000')

def get_weekend_index(date_range):
    res = []
    for idx, d in enumerate(date_range):
        if d.weekday() in (5, 6):
            res.append(idx)
    return res

class Entity:
    def __init__(self) -> None:
        pass
    
    @property
    def column_names(self) -> list:
        return self._column_names
    
    @column_names.setter
    def column_names(self, value: list) -> None:
        self._column_names = value
    
    @property
    def column_dtypes(self) -> dict:
        return self._column_dtypes
    
    @column_dtypes.setter
    def column_dtypes(self, value: dict) -> None:
        self._column_dtypes = value

    def load_file(self, f: str, type: str='excel', engine: str='openpyxl') -> None:
        if type == 'excel':
            self._ori_data = pd.read_excel(f, names=self.column_names, dtype=self.column_dtypes, engine=engine)
        elif type == 'test':
            self._ori_data = pd.DataFrame()
        
        self._preprocess()
        self._process()
    
    def _preprocess(self) -> None:
        pass
    
    def _process(self) -> None:
        pass

class COrder(Entity):
    def __init__(self) -> None:
        super().__init__()
        self.column_names = [
            "充电订单号", "充电完成状态", "用户编码", "用户昵称", "手机号【加密】",
            "运营商名称", "充电站名称", "充电枪编号", "订单电量(度)", "订单总金额(元)",
            "订单电费(元)", "订单服务费(元)", "实收金额", "点数卡券id", "点数卡对外展示名称",
            "点数卡名称", "点数卡抵扣金额（元）", "点数卡抵扣点数", "异常代码/原因", "订单创建日期",
            "充电完成日期", "支付完成日期", "订单创建时间", "充电完成时间", "支付完成时间",
            "运营商id", "充电站id",
        ]

        dtypes = dict.fromkeys(self.column_names, np.str0)
        dtypes["订单电量(度)"] = np.float32
        dtypes["订单总金额(元)"] = np.float32
        dtypes["订单服务费(元)"] = np.float32
        dtypes["订单电费(元)"] = np.float32
        dtypes["实收金额"] = np.float32
        dtypes["点数卡抵扣金额（元）"] = np.float32
        dtypes["点数卡抵扣点数"] = np.float32
        dtypes["订单创建日期"] = np.datetime64
        dtypes["充电完成日期"] = np.datetime64
        dtypes["支付完成日期"] = np.datetime64
        dtypes["订单创建时间"] = np.datetime64
        dtypes["充电完成时间"] = np.datetime64
        dtypes["支付完成时间"] = np.datetime64
        self.column_dtypes = dtypes
        
        self.min_d = None
        self.max_d = None
    
    def _preprocess(self) -> None:
        # nan字段补零
        self._ori_data["实收金额"].fillna(0, inplace=True)
        self._ori_data["点数卡抵扣金额（元）"].fillna(0, inplace=True)
        self._ori_data["点数卡抵扣点数"].fillna(0, inplace=True)
        # drop零订单
        self._ori_data.drop(index=self._ori_data[self._ori_data["订单电量(度)"] == 0].index, inplace=True)
    
    def _process(self) -> None:
        self.min_d = self._ori_data["充电完成日期"].min().date()
        self.max_d = self._ori_data["充电完成日期"].max().date()
    
    # 度电分布
    def plot_kWhs_dist_by_order(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, bins: int=50, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date))][["订单电量(度)"]].copy()

        ax.hist(df_copy["订单电量(度)"], bins=bins)
        mean = df_copy["订单电量(度)"].mean()
        ax.axvline(x=mean, color='green', linestyle='--')
        ax.set_xlabel("订单度电数(度)")
        ax.set_ylabel("订单数(个)")
        ax.set_title("订单度电数分布(%s - %s))\n均值 %.2f度" % (from_date, to_date, mean))
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')

    # 费用分布
    def plot_total_fee_dist_by_order(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, bins: int=50, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date))][["订单总金额(元)"]].copy()
        
        ax.hist(df_copy["订单总金额(元)"], bins=bins)
        mean = df_copy["订单总金额(元)"].mean()
        ax.axvline(x=mean, color='green', linestyle='--')
        ax.set_xlabel("订单总金额(元)")
        ax.set_ylabel("订单数(个)")
        ax.set_title("订单总金额分布(%s - %s))\n均值 %.2f元" % (from_date, to_date, mean))
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')     

    # 单价分布
    def plot_unit_price_dist_by_order(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, bins: int=50, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date))][["订单电量(度)", "订单总金额(元)"]].copy()
        df_copy.loc[:, "度电单价(元)"] = df_copy.loc[:, "订单总金额(元)"] / df_copy.loc[:, "订单电量(度)"]
        
        ax.hist(df_copy["度电单价(元)"], bins=bins)
        mean = df_copy["度电单价(元)"].mean()
        ax.axvline(x=mean, color='green', linestyle='--')
        ax.set_xlabel("度电单价(元)")
        ax.set_ylabel("订单数(个)")
        ax.set_title("度电单价分布(%s - %s))\n均值 %.3f元" % (from_date, to_date, mean))
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')

    # 充电时段分布
    def plot_charge_dist_by_hour(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date)) \
            & (self._ori_data["充电完成时间"] > self._ori_data["订单创建时间"])][["充电完成时间"]].copy()
        df_copy.loc[:, "充电完成时段"] = df_copy.loc[:, "充电完成时间"].dt.hour
        
        ax.hist(df_copy["充电完成时段"], bins=24, edgecolor='w')
        ax.set_xticks(np.linspace(0, 23, 24))
        ax.set_xlabel("充电完成时段(点)")
        ax.set_ylabel("订单数(个)")
        ax.set_title("充电时段分布(%s - %s)" % (from_date, to_date))
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')

    # 趋势 - 总金额|电费|服务费
    def plot_total_fee_trend_1_by_day(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date))]\
            .groupby("充电完成日期", as_index=False)[["订单总金额(元)", "订单电费(元)", "订单服务费(元)"]].sum().copy()

        # 补空缺日期
        date_range = pd.date_range(start=from_date, end=to_date, freq='D')
        df_copy = df_copy.set_index("充电完成日期").reindex(pd.Index(date_range, name="充电完成日期"), fill_value=0).reset_index()
        
        ax.plot(df_copy["充电完成日期"], df_copy["订单总金额(元)"], label='订单总金额(元)')
        ax.plot(df_copy["充电完成日期"], df_copy["订单电费(元)"], label='订单电费(元)')
        ax.plot(df_copy["充电完成日期"], df_copy["订单服务费(元)"], label='订单服务费(元)')
        ax.set_xlabel("充电完成日期")
        ax.set_ylabel("金额(元)")
        ax.set_xticks(date_range)
        for idx in get_weekend_index(date_range):
            ax.get_xticklabels()[idx].set_color("grey")
        ax.set_title('总金额/电费/服务费趋势(%s - %s)' % (from_date, to_date))
        ax.legend()
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')

    # 趋势 - 总金额|实收|权益抵扣
    def plot_total_fee_trend_2_by_day(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date))]\
            .groupby("充电完成日期", as_index=False)[["订单总金额(元)", "实收金额", "点数卡抵扣金额（元）"]].sum().copy()

        # 补空缺日期
        date_range = pd.date_range(start=from_date, end=to_date, freq='D')
        df_copy = df_copy.set_index("充电完成日期").reindex(pd.Index(date_range, name="充电完成日期"), fill_value=0).reset_index()
        
        ax.plot(df_copy["充电完成日期"], df_copy["订单总金额(元)"], label='订单总金额(元)')
        ax.plot(df_copy["充电完成日期"], df_copy["实收金额"], label='实收金额(元)')
        ax.plot(df_copy["充电完成日期"], df_copy["点数卡抵扣金额（元）"], label='点数卡抵扣金额(元)')
        ax.set_xlabel("充电完成日期")
        ax.set_ylabel("金额(元)")
        ax.set_xticks(date_range)
        for idx in get_weekend_index(date_range):
            ax.get_xticklabels()[idx].set_color("grey")
        ax.set_title('总金额/实收/权益抵扣分析(%s - %s)' % (from_date, to_date))
        ax.legend()
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')
    
    # 趋势 - 订单度电|权益点数抵扣
    def plot_total_fee_trend_3_by_day(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date))]\
            .groupby("充电完成日期", as_index=False)[["订单电量(度)", "点数卡抵扣点数"]].sum().copy()

        # 补空缺日期
        date_range = pd.date_range(start=from_date, end=to_date, freq='D')
        df_copy = df_copy.set_index("充电完成日期").reindex(pd.Index(date_range, name="充电完成日期"), fill_value=0).reset_index()
        
        ax.plot(df_copy["充电完成日期"], df_copy["订单电量(度)"], label='订单电量(度)')
        ax.plot(df_copy["充电完成日期"], df_copy["点数卡抵扣点数"], label='点数卡抵扣点数(度)')
        ax.set_xlabel("充电完成日期")
        ax.set_ylabel("电量(度)")
        ax.set_xticks(date_range)
        for idx in get_weekend_index(date_range):
            ax.get_xticklabels()[idx].set_color("grey")
        ax.set_title('总度电数/权益点数抵扣趋势(%s - %s)' % (from_date, to_date))
        ax.legend()
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')
    
    # 趋势 - 订单度电，按运营商分解
    def plot_total_fee_trend_4_by_day(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d

        df_copy = self._ori_data[(self._ori_data["充电完成日期"] >= pd.to_datetime(from_date)) & (self._ori_data["充电完成日期"] <= pd.to_datetime(to_date))]\
            .groupby(["运营商名称", "充电完成日期"], as_index=False)[["订单电量(度)"]].sum().copy()

        # 补空缺日期
        date_range = pd.date_range(start=from_date, end=to_date, freq='D')
        mux = pd.MultiIndex.from_product([df_copy["运营商名称"].unique(), date_range], names=["运营商名称", "充电完成日期"])
        df_copy = df_copy.set_index(["运营商名称", "充电完成日期"]).reindex(mux, fill_value=0).reset_index()

        tmp = np.zeros(shape=((to_date - from_date).days + 1))
        for k in df_copy["运营商名称"].unique():
            d = df_copy[df_copy["运营商名称"] == k]
            ax.bar(d["充电完成日期"], d["订单电量(度)"], label=k, bottom=tmp, color=[get_color_style(k)])
            tmp += d["订单电量(度)"].to_numpy()
        # del tmp
        ax.set_xlabel('充电完成日期')
        ax.set_ylabel('订单度电数(度)')
        ax.set_xticks(date_range)

        for idx in get_weekend_index(date_range):
            ax.get_xticklabels()[idx].set_color("grey")

        ax.set_title('总度电数按运营商分解趋势(%s - %s)' % (from_date, to_date))
        ax.legend()
        if savefig:
            plt.savefig(file, dpi=dpi, bbox_inches='tight')
    
    def get_overdue_payment(self, file: str, overdue_days:int = 7, savecsv: bool=True) -> pd.DataFrame:
        df_copy = self._ori_data[(self._ori_data["支付完成日期"].isna()) & (self._ori_data["充电完成日期"].isna() == False)]\
            [["充电订单号", "用户编码", "用户昵称", "手机号【加密】", "运营商名称", 
              "充电站名称", "订单电量(度)", "订单总金额(元)", "订单电费(元)", "订单服务费(元)", 
              "订单创建日期", "充电完成日期"]].copy()
        df_copy.set_index("充电订单号", inplace=True)
        df_copy.loc[:, "逾期天数"] = pd.to_datetime(dt.date.today()) - df_copy.loc[:, "充电完成日期"]
        df_copy = df_copy[df_copy["逾期天数"] >= pd.Timedelta(overdue_days, 'D')]
        df_copy.sort_values(["充电完成日期"], inplace=True)
        if savecsv:
            df_copy.to_csv(file, encoding='utf_8_sig', sep=',')
        return df_copy

class User(Entity):
    def __init__(self) -> None:
        super().__init__()
        self.column_names = [
            "用户id", "用户昵称", "vin", "车系名称", # "交车城市", 
            "交车日期", # "用户首次充电订单完成日期",
            "用户首次充电订单号", "交车时间", "车型版本",
            "车主id", "用户手机"
        ]

        dtypes = dict.fromkeys(self.column_names, np.str0)
        dtypes["交车日期"] = np.datetime64
        # dtypes["用户首次充电订单完成日期"] = np.datetime64
        dtypes["交车时间"] = np.datetime64
        self.column_dtypes = dtypes
        
        self.min_d = None
        self.max_d = None
    
    def _preprocess(self) -> None:
        # drop 交车日期、车系名称 为空的记录
        self._ori_data.dropna(how='any', subset=["交车日期", "车系名称"], inplace=True)
    
    def _process(self) -> None:
        self.min_d = self._ori_data["交车日期"].min().date()
        self.max_d = self._ori_data["交车日期"].max().date()

    def plot_car_delivery_trend_by_day(self, from_date: dt.date, to_date: dt.date, ax: Axes, file: str, savefig: bool=True, dpi: int=100) -> None:
        if from_date == None:
            from_date = self.min_d
        if to_date == None:
            to_date = self.max_d
        
        df_copy = self._ori_data[(self._ori_data["交车日期"] >= pd.to_datetime(from_date)) & (self._ori_data["交车日期"] <= pd.to_datetime(to_date))]\
                                 [["vin", "交车日期"]].drop_duplicates().copy()
            
        df_copy = df_copy.groupby("交车日期", as_index=False).agg(交车数量=('vin', 'count'))
        date_range = pd.date_range(from_date, to_date, freq='D')
        df_copy = df_copy.set_index("交车日期").reindex(pd.Index(date_range, name="交车日期"), fill_value=0).reset_index()

        ax.bar(df_copy["交车日期"], df_copy["交车数量"], label='交车数量(台)')
        ax.set_xlabel("交车日期")
        ax.set_ylabel("交车数量(台)")
        ax.set_xticks(date_range)
        for idx in get_weekend_index(date_range):
            ax.get_xticklabels()[idx].set_color("grey")
        ax.set_title('交车数量(%s - %s)' % (from_date, to_date))
        ax.legend()
        if savefig:
            plt.savefig(file, dpi=100, bbox_inches='tight')


In [2]:
# generate pptx from template
from pptx import Presentation
from pptx.slide import Slide
from pptx.shapes.placeholder import PicturePlaceholder, SlidePlaceholder, TablePlaceholder, PlaceholderPicture
from pptx.util import Pt

class PowerPointGenerator:
    def __init__(self, template_file: str, conf: list, from_date: dt.date, to_date: dt.date) -> None:
        self._prs = Presentation(template_file)
        self.conf = conf
        self.min_d = from_date
        self.max_d = to_date
    
    def _adjust_picture_to_fit(self, ph: PlaceholderPicture) -> None:
        # ph - PicturePlaceholder
        
        l = ph.left
        t = ph.top
        w = ph.width
        h = ph.height

        ph_pic_width = ph.width
        ph_pic_height = ph.height
        (img_width, img_height) = ph.image.size
        ph_aspect_ratio = float(ph_pic_width) / float(ph_pic_height)
        img_aspect_ratio = float(img_width) / float(img_height)

        ph.crop_top = 0
        ph.crop_left = 0
        ph.crop_bottom = 0
        ph.crop_right = 0

        # if the placeholder is "wider" in aspect, shrink the picture width while maintaining the image aspect ratio
        if ph_aspect_ratio > img_aspect_ratio:
            ph.width = int(img_aspect_ratio * ph_pic_height)
            ph.height = ph_pic_height
        # otherwise shrink the height
        else:
            ph.width = ph_pic_width
            ph.height = int(ph_pic_width / img_aspect_ratio)
        
        ph.left = l + int((w - ph.width) / 2)
        ph.top = t + int((h - ph.height) / 2)
    
    def _insert_picture(self, ph: PicturePlaceholder, file: str) -> None:
        pic = ph.insert_picture(file)
        self._adjust_picture_to_fit(pic)

    def _insert_table(self, ph: TablePlaceholder, file: str, col_filter: list, font_size: int=11) -> None:
        df = pd.read_csv(file)
        df = df[col_filter]
        gf = ph.insert_table(rows=len(df)+1, cols=len(df.columns))
        tb = gf.table
        # table titles
        for c in range(len(df.columns)):
            tb.cell(0, c).text = df.columns[c]
            tb.cell(0, c).text_frame.paragraphs[0].font.size = Pt(font_size)
        # table contents
        for r in range(len(df)):
            for c in range(len(df.columns)):
                tb.cell(r+1, c).text = str(df.iloc[r, c])
                tb.cell(r+1, c).text_frame.paragraphs[0].font.size = Pt(font_size)

    def _insert_text(self, ph: SlidePlaceholder, text: str):
        ph.text = text

    def gen_slides(self, input_dir: str) -> None:
        for idx in range(len(self.conf)):
            slide = self._prs.slides[idx]
            self._gen_slide(input_dir, slide, idx)

    def _gen_slide(self, input_dir: str, slide, idx) -> None:
        for ph in slide.placeholders:
            if isinstance(ph, TablePlaceholder):
                if 'TablePlaceholder' in self.conf[idx].keys():
                    if ph.placeholder_format.idx in self.conf[idx]['TablePlaceholder'].keys():
                        self._insert_table(ph, os.path.join(input_dir, self.conf[idx]['TablePlaceholder'][ph.placeholder_format.idx][0]), self.conf[idx]['TablePlaceholder'][ph.placeholder_format.idx][1])
            elif isinstance(ph, PicturePlaceholder):
                if 'PicturePlaceholder' in self.conf[idx].keys():
                    if ph.placeholder_format.idx in self.conf[idx]['PicturePlaceholder'].keys():
                        self._insert_picture(ph, os.path.join(input_dir, self.conf[idx]['PicturePlaceholder'][ph.placeholder_format.idx]))
            elif isinstance(ph, SlidePlaceholder):
                if 'SlidePlaceholder' in self.conf[idx].keys():
                    if ph.placeholder_format.idx in self.conf[idx]['SlidePlaceholder'].keys():
                        self._insert_text(ph, self.conf[idx]['SlidePlaceholder'][ph.placeholder_format.idx])

    def save(self, file: str) -> None:
        self._prs.save(file)

In [3]:
# if __name__ == "__main__":
today_date = dt.date.today()
from_date = dt.date(2023, 3, 1)
to_date = today_date + dt.timedelta(days=-1)

root_dir = r'D:\自动生成报表'
output_dir = os.path.join(root_dir, '%s' % today_date)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

c_order_file = os.path.join(root_dir, r'充电桩_战略_公共充电订单(C端).xlsx')
user_file = os.path.join(root_dir, r'充电桩_战略_用户及车全量信息.xlsx')

# toc order
c_order = COrder()
c_order.load_file(c_order_file, type='excel')

ax = plt.figure(figsize=(4, 4)).gca()
c_order.plot_kWhs_dist_by_order(from_date, to_date, ax, os.path.join(output_dir, '11.分布_电量_订单.png'))
plt.close()

ax = plt.figure(figsize=(4, 4)).gca()
c_order.plot_total_fee_dist_by_order(from_date, to_date, ax, os.path.join(output_dir, '12.分布_总金额_订单.png'))
plt.close()

ax = plt.figure(figsize=(4, 4)).gca()
c_order.plot_unit_price_dist_by_order(from_date, to_date, ax, os.path.join(output_dir, '13.分布_单价_订单.png'))
plt.close()

ax = plt.figure(figsize=(4, 4)).gca()
c_order.plot_charge_dist_by_hour(from_date, to_date, ax, os.path.join(output_dir, '14.分布_订单数_小时.png'))
plt.close()

ax = plt.figure(figsize=(20, 4)).gca()
c_order.plot_total_fee_trend_1_by_day(from_date, to_date, ax, os.path.join(output_dir, '21.趋势_总金额_电费_服务费_日期.png'))
plt.close()

ax = plt.figure(figsize=(20, 4)).gca()
c_order.plot_total_fee_trend_2_by_day(from_date, to_date, ax, os.path.join(output_dir, '22.趋势_总金额_实收权益抵扣_日期.png'))
plt.close()

ax = plt.figure(figsize=(20, 4)).gca()
c_order.plot_total_fee_trend_3_by_day(from_date, to_date, ax, os.path.join(output_dir, '23.趋势_订单度电_权益点数抵扣_日期.png'))
plt.close()

ax = plt.figure(figsize=(20, 4)).gca()
c_order.plot_total_fee_trend_4_by_day(from_date, to_date, ax, os.path.join(output_dir, '24.趋势_订单度电_按运营商分解_日期.png'))
plt.close()

_ = c_order.get_overdue_payment(os.path.join(output_dir, '41.财务_超期7天支付.csv'), overdue_days=7)

# user
user = User()
user.load_file(user_file, type='excel')

ax = plt.figure(figsize=(20, 4)).gca()
user.plot_car_delivery_trend_by_day(from_date, to_date, ax, os.path.join(output_dir, '31.趋势_新车交付数_日期.png'))
plt.close()

In [4]:
slides_conf = [{
            'SlidePlaceholder': {
                10: r'公共充电业务日报',
                11: r'趋势时间周期 (%s - %s)' % (from_date, to_date),
                12: r'%s' % dt.date.today(),
            }
        }, {
            'PicturePlaceholder': {
                32: r'11.分布_电量_订单.png',
                55: r'12.分布_总金额_订单.png',
                56: r'13.分布_单价_订单.png'
            }
        }, {
            'PicturePlaceholder': {
                32: r'14.分布_订单数_小时.png',
            }
        }, {
            'PicturePlaceholder': {
                43: r'24.趋势_订单度电_按运营商分解_日期.png',
            }
        }, {
            'PicturePlaceholder': {
                43: r'23.趋势_订单度电_权益点数抵扣_日期.png',
            }
        }, {
            'PicturePlaceholder': {
                43: r'22.趋势_总金额_实收权益抵扣_日期.png',
            }
        }, {
            'PicturePlaceholder': {
                43: r'21.趋势_总金额_电费_服务费_日期.png',
            }
        }, {
            'TablePlaceholder': {
                15: (r'41.财务_超期7天支付.csv', ["充电订单号", "运营商名称", "订单总金额(元)", "充电完成日期", "逾期天数",])
            }
        }, {
        }]

ppt = PowerPointGenerator(os.path.join(root_dir, '公共充电业务日报模板.pptx'), slides_conf, from_date, to_date)
ppt.gen_slides(os.path.join(root_dir, '%s' % today_date))
ppt.save(os.path.join(output_dir, '公共充电业务日报.pptx'))